In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import cftime
import yaml
import xarray as xr
import rpy2.robjects as ro

#My functions
sys.path.insert(0,'../functions/')
import functions_HeatWavesCities as fun_HWC


In [ ]:
#Read main path
with open('../path_main.txt', 'r') as file:   path_main = file.read()
    
dir_CORDEX    = f'{path_main}Data/EURO-CORDEX/'
dir_data      = f'{path_main}Data/EURO-CORDEX/Variables/'
dir_scripts   = f'{path_main}Scripts/'
dir_names     = f'{path_main}Scripts/Model_lists/'
dir_functions = f'{path_main}Scripts/functions/'
dir_HSI_out   = f'{path_main}Data/EURO-CORDEX/HSIs/'
if not os.path.exists(dir_HSI_out):   os.mkdir(dir_HSI_out)


## Prepare variables and parameters

In [ ]:
#Source R functions
r = ro.r
r.source(dir_scripts + 'functions/get_HWMId_vCities_NxN.r')

#Define cities
cities = ['Istanbul', 'Moscow', 'London', 'SaintPetersburg', 'Berlin', 'Madrid', 'Kyiv', 'Rome', 'Paris',
          'Bucharest', 'Minsk', 'Vienna', 'Hamburg', 'Warsaw', 'Budapest', 'Barcelona', 'Munich', 'Kharkiv',
          'Milan', 'Belgrade', 'Prague', 'NizhnyNovgorod', 'Kazan', 'Sofia', 'Brussels', 'Stockholm', 'Oslo',
          'Dublin', 'Lisbon', 'Vilnius', 'Copenhagen', 'Helsinki', 'Athens', 'Amsterdam', 'Riga', 'Zagreb']

#Define HSIs
HSI_names = ['TN', 'TX']

N = 5
N_str = str(N) + 'x' + str(N)

#Define RCPs
RCPs = ['rcp85']

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp26'] = []
all_models['rcp85'] = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP26.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp26'].append(eval(line[:-1]))
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(eval(line[:-1]))

#Add models for historical
mod_85 = ["_".join(model) for model in all_models['rcp85']]
mod_26 = ["_".join(model) for model in all_models['rcp26']]
all_models['historical'] = [model.split('_') for model in sorted(list(set(mod_85).union(set(mod_26))))]


## HWMId for NxN grid points

In [ ]:
#Select NxN grid points
area_sel = N_str

#Only performs calculation for gridpoint, otherwise use script 'Prep_HSI_HWMId_EURO-CORDEX'
if area_sel=='':
    sys.exit('Use script Prep_HSI_HWMId_EURO-CORDEX for data on on a single gripdoint.')

#Define output folder
dir_HWMId_out = dir_CORDEX + 'HWMId_' + N_str + '/'
if not os.path.exists(dir_HWMId_out): os.mkdir(dir_HWMId_out)
    
#Define reference and application years
app_years = [1981, 2099]
ref_years_vec = ro.r.c(1981, 2010)
app_years_vec = ro.r.seq(app_years[0], app_years[1])

#Create time vector for HWMId
time_HWMID = xr.cftime_range(start=str(app_years[0]) + '0101', end=str(app_years[1]) + '1231', freq='Y', calendar='standard')

#Loop over RCPs
for RCP in RCPs:
    
    #Define scenarios and select models
    scenarios = ['historical', RCP]
    models    = all_models[RCP]

    #Loop over models
    for model in models:
        
        print('')
        print(" -" + "_".join(model), end=': ')

        #Loop over cities
        for i0, city in enumerate(cities):

            print(city, end=', ')
            
            #Define and create output directory
            dir_city = dir_HWMId_out + city + '/'
            dir_out  = dir_HWMId_out + city + '/' + RCP + '/'
            if not os.path.exists(dir_city): os.mkdir(dir_city)
            if not os.path.exists(dir_out): os.mkdir(dir_out)
 
            #Loop over HSIs
            for HSI_name in HSI_names:
                
                #Loop over scenarios
                for i1, scen in enumerate(scenarios):

                    #Get file name
                    dir_HSI = dir_HSI_out + city + '/' + scen + '/'
                    var_data = HSI_name
                    files_HSI = [file for file in os.listdir(dir_HSI) if area_sel + '_' in file and scen in file and model[0] in file and model[1] in file and model[2] in file]
                    if len(files_HSI)!=1:  sys.exit('File is not unique')
                        
                    #Read data
                    data_read = xr.open_dataset(dir_HSI + files_HSI[0])
                    data_read = data_read[var_data].to_dataset(name=HSI_name)

                    #Concatenate historical and future data
                    if i1==0:
                        data_comb = data_read
                    else:
                        data_comb = xr.concat((data_comb, data_read), dim='time')
                
                #Save data temporarily in NetCDF
                fname_tmp   = dir_HSI + 'tmp_EURO-CORDEX_' + area_sel + '.nc'
                if os.path.exists(fname_tmp): os.remove(fname_tmp)
                data_comb.to_netcdf(fname_tmp)

                #Define lon and lat name
                if 'rlat' in data_comb.dims:
                    coord_names = ['rlon', 'rlat']
                    coord_names_R = ro.r.c('rlon', 'rlat')
                elif 'x' in data_comb.dims:
                    coord_names = ['y', 'x']
                    coord_names_R = ro.r.c('y', 'x')
                    
                #Calculate HWMId
                output = r.get_HWMId_vCities_NxN(HSI_name, fname_tmp, ref_years_vec, app_years_vec, coord_names_R, dir_functions)

                #Extract variables from R output (dims: lon, lat, time)
                HWMID_strength = np.array(output.rx2("HWMID_strength"))
                HWMID_length   = np.array(output.rx2("HWMID_length"))
                HWMID_DOYstart = np.array(output.rx2("HWMID_DOYstart"))

                #Create dataset with same coords as original dataset
                data_HWMID = xr.Dataset(coords=dict(zip(data_comb.coords, [data_comb[coord] for coord in data_comb.coords])))
                if 'height' in data_HWMID.coords: data_HWMID = data_HWMID.drop('height')
                
                #Put yearly time axis
                data_HWMID = data_HWMID.isel(time=1).drop('time')
                data_HWMID = data_HWMID.assign_coords({'time': time_HWMID})

                #Put HWMID
                data_HWMID['HWMID'] = (coord_names + ['time'], HWMID_strength)
                data_HWMID['HWMID_length'] = (coord_names + ['time'], HWMID_length)
                data_HWMID['HWMID_DOYstart'] = (coord_names + ['time'], HWMID_DOYstart)
                
                #Save in file
                fname_HWMId = dir_out + 'HWMId-' + HSI_name + "_" + "_".join(model) + '_' + "-".join(scenarios) + "_" + str(app_years[0]) + "-" + str(app_years[1]) + ".nc"
                if os.path.exists(fname_HWMId): os.remove(fname_HWMId)
                data_HWMID.to_netcdf(fname_HWMId)

                #Remove temporary file
                os.remove(fname_tmp)
                